# 🏥 ZP Market Monitoring - Daily Crawl & Rank

## 목적
**에이전시 결과와 비교 검증**
- 전날(어제) 1일치 기사만 크롤링
- Weekly와 동일한 LGBM 모델로 랭킹
- 에이전시가 선별한 결과와 우리 모델 결과 비교

## 실행 방법
1. 아래 셀들을 **순서대로** 실행
2. 결과 파일 다운로드
3. 대시보드에서 에이전시 결과와 비교

**예상 소요 시간: 3-5분**

## Step 1: 환경 설정

In [ ]:
# 필요한 패키지 설치
!pip install -q konlpy requests beautifulsoup4 pandas pyyaml scikit-learn lightgbm
print("✅ 패키지 설치 완료!")

## Step 2: 프로젝트 다운로드

In [ ]:
# GitHub에서 프로젝트 클론
!rm -rf zp-market-monitoring-v3-share
!git clone https://github.com/Lilyleeyaa/zp-market-monitoring-v3-share.git
%cd zp-market-monitoring-v3-share
print("✅ 프로젝트 다운로드 완료!")

## Step 3: Daily 크롤링 (어제 1일치)

In [ ]:
# Daily 크롤링 실행 (config/keywords.yaml의 daily 키워드 사용)
!python scripts/crawl_daily_news.py
print("\n✅ Daily 크롤링 완료!")
print("결과 파일: data/articles_raw/articles_daily_YYYYMMDD.csv")

## Step 4: LGBM 모델 랭킹 (Weekly와 동일 모델)

In [ ]:
# Daily 파일에 대해 모델 랭킹 적용
import os
import glob
from datetime import datetime, timedelta

# 어제 날짜로 파일명 생성
yesterday = (datetime.now() - timedelta(days=1)).strftime('%Y%m%d')
input_file = f"data/articles_raw/articles_daily_{yesterday}.csv"
output_file = f"data/articles_raw/articles_daily_ranked_{yesterday}.csv"

# rank_articles.py 실행
!python scripts/rank_articles.py

# 출력 파일이 articles_ranked_*.csv로 생성되었을 경우 이름 변경
ranked_files = sorted(glob.glob("data/articles_raw/articles_ranked_*.csv"))
if ranked_files:
    latest_ranked = ranked_files[-1]
    # Daily용으로 이름 변경
    if 'daily' not in latest_ranked:
        os.rename(latest_ranked, output_file)
        print(f"\n✅ 파일명 변경: {output_file}")

print("\n✅ 모델 랭킹 완료!")
print(f"최종 결과: {output_file}")

## Step 5: 결과 파일 다운로드

In [ ]:
# Daily 랭킹 파일 다운로드
from google.colab import files
from datetime import datetime, timedelta
import os

yesterday = (datetime.now() - timedelta(days=1)).strftime('%Y%m%d')
output_file = f"data/articles_raw/articles_daily_ranked_{yesterday}.csv"

if os.path.exists(output_file):
    print(f"📄 Daily 랭킹 파일: {output_file}")
    files.download(output_file)
    print("\n✅ 파일 다운로드 완료!")
    print("\n다음 단계:")
    print("1. 다운로드한 파일을 로컬 프로젝트 data/articles_raw/ 폴더에 복사")
    print("2. Git add, commit, push")
    print("3. Daily Validation 대시보드에서 에이전시 결과와 비교")
else:
    print(f"❌ 파일을 찾을 수 없습니다: {output_file}")
    print("위 셀들을 다시 실행해주세요.")

---
## 🎉 완료!

### 에이전시 비교 방법:
1. 에이전시에서 받은 Daily 결과 CSV 준비
2. Daily Validation 대시보드 접속
3. 두 결과 비교:
   - 키워드별 기사 수
   - 중복 기사
   - 모델 점수 분포
   - Top N 기사 일치율

내일 다시 실행하세요!